In [1]:
import pybullet as p
import pybullet_data as pd
import math
import time
import numpy as np
import panda as panda_sim


In [2]:

p.connect(p.GUI)
p.setAdditionalSearchPath(r'C:\Users\vase_\Downloads\robot-simulation')
p.resetDebugVisualizerCamera( cameraDistance=10, cameraYaw=15, cameraPitch=-20, cameraTargetPosition=[0,0,0])

In [3]:
orn=[0, 0.0, 0.0, 1]
flags = p.URDF_ENABLE_CACHED_GRAPHICS_SHAPES
p.loadURDF("outfile.urdf", np.array([0,0,0]), orn, useFixedBase=True, flags=flags)

0

In [4]:
parameters = []
joints = []
for i in range(p.getNumJoints(0)):
    joint_info = p.getJointInfo(0,i)
    if joint_info[2] == p.JOINT_PRISMATIC:
        joints.append(i)
        parameters.append((False,p.addUserDebugParameter("D"+str(i),joint_info[8],joint_info[9],0.5)))
    if joint_info[2] == p.JOINT_REVOLUTE:
        joints.append(i)
        parameters.append((True,p.addUserDebugParameter("Theta"+str(i),np.rad2deg(joint_info[8]),np.rad2deg(joint_info[9]),0)))

In [18]:
file1 = open('outfile.urdf', 'r')
Lines = file1.readlines()
  
count = 0
# Strips the newline character
for line in Lines[1:]:
    if line.strip() == '-->':
        break
    dhlist = line.strip().split(',')
    print(dhlist[0])
    if(dhlist[0][1] == 'p'):
        print('ok')

    

'r'
'r'
'p'
ok
'f'
'r'


In [10]:
p.JOINT_REVOLUTE

0

In [10]:
p.getLinkState(0,2,computeForwardKinematics=1) 

((2.0, 0.0, 3.0),
 (0.0, 0.0, 0.0, 1.0),
 (0.0, 0.0, 0.0),
 (0.0, 0.0, 0.0, 1.0),
 (2.0, 0.0, 3.0),
 (0.0, 0.0, 0.0, 1.0))

In [6]:
p.getMatrixFromQuaternion((0.0, 0.0, 0.0, 1.0)) 

(1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0)

In [13]:
p.addUserDebugLine((2, 0, 0),(2, 0.0, 1.0),parentObjectUniqueId=0,parentLinkIndex=2)

4

In [11]:
p.addUserDebugLine((2, 0, 3),(2, 0.0, 4.0))

3

In [5]:
timeStep=1./60.
p.setTimeStep(timeStep)
p.setGravity(0,-9.8,0)
 
while (1):
	p.stepSimulation()
	targets = [(np.deg2rad(p.readUserDebugParameter(parameter)) if revolute else p.readUserDebugParameter(parameter)) for revolute,parameter in parameters]
	p.setJointMotorControlArray(0,joints,p.POSITION_CONTROL,targets)
	time.sleep(timeStep)

error: Not connected to physics server.

In [11]:
import sympy as sp

In [14]:
def frame_lines(pose, line_length=1):
    t = pose[:3, -1]
    line_n = sp.Matrix.hstack(t, t + line_length * pose[:3, 0])
    line_o = sp.Matrix.hstack(t, t + line_length * pose[:3, 1])
    line_a = sp.Matrix.hstack(t, t + line_length * pose[:3, 2])
    return line_n, line_o, line_a


def draw_frame(pose, labels='x-y-z', colors=('red', 'green', 'blue'), line_width=3, line_length=1):
    def go_scatter(x, y, z, name, color, width):
        return go.Scatter3d(
            x=x, y=y, z=z,
            marker=dict(size=2),
            line=dict(color=color, width=width),
            name=name)
    
    labels = labels.split('-')
    line_n, line_o, line_a = frame_lines(pose, line_length)
    scatter_n = go_scatter(*line_n.tolist(), labels[0], colors[0], line_width)
    scatter_o = go_scatter(*line_o.tolist(), labels[1], colors[1], line_width)
    scatter_a = go_scatter(*line_a.tolist(), labels[2], colors[2], line_width)
    return scatter_n, scatter_o, scatter_a

In [19]:
line_n, line_o, line_a = frame_lines(pose)

In [21]:
line_a

Matrix([
[4.0,            4.0],
[  0, -1.2246468e-16],
[3.0,            2.0]])

In [17]:
pose =  sp.Matrix([[ 1.0000000e+00,  0.0000000e+00,  0.0000000e+00,  4.0000000e+00],
       [ 0.0000000e+00, -1.0000000e+00, -1.2246468e-16,  0.0000000e+00],
       [ 0.0000000e+00,  1.2246468e-16, -1.0000000e+00,  3.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  1.0000000e+00]])

In [18]:
pose

Matrix([
[1.0,             0,              0, 4.0],
[  0,          -1.0, -1.2246468e-16,   0],
[  0, 1.2246468e-16,           -1.0, 3.0],
[  0,             0,              0, 1.0]])

In [3]:
p.setAdditionalSearchPath(pd.getDataPath())

In [ ]:
timeStep=1./60.
p.setTimeStep(timeStep)
p.setGravity(0,-9.8,0)
 
panda = panda_sim.PandaSim(p,[0,0,0])
while (1):
	panda.step()
	p.stepSimulation()
	time.sleep(timeStep)